# Chapter 4 - Bedrock APIs: Car Damage Assessment with Claude 3

## Overview
This notebook demonstrates how to use Amazon Bedrock with Claude 3 for computer vision tasks, specifically car damage assessment. We'll explore how to process images and generate detailed damage reports using foundation models.

## Introduction
This notebook demonstrates how to use foundation models with multimodal capabilities to automate insurance claims processing. We'll utilize Claude 3 Sonnet's vision capabilities to analyze car damage from images and generate detailed assessments that could streamline the claims process.

## Prerequisites
- AWS account with Amazon Bedrock access
- Access to Claude 3 Sonnet multimodal model
- Car damage image for assessment

## Setup

### Install Required Dependencies

In [ ]:
# Install required packages
%pip install -U --no-cache-dir boto3

### Import Libraries

In [42]:
# 📦 Import libraries
import json
import os
import sys

import boto3
import botocore

# 🔧 Initialize Bedrock client
boto3_bedrock = boto3.client('bedrock-runtime')

## Image Processing

### Load and Encode Image

In [ ]:
# 🖼️ Import base64 for image encoding
import base64

# 📁 Load and encode image as base64
with open("cardamage.jpg", "rb") as f:
    encoded_image = base64.b64encode(f.read()).decode('utf-8')

## Define Assessment Prompt

### Create Structured Prompt

In [44]:
# 💬 Define damage assessment prompt
prompt = """
You are an assistant to insurance claims agents. Your job is to provide an assessment in three categories - minor damage, moderate damage, and major damage. You can provide additional details such as what part of the vehicle is damage, what kind of repair will likely be require, and approximate average cost. Be reasonable, accurate, and do not inflate or deflate the extent of damage. 

Please provide your assessment for the supplied image.
"""

## Build API Request

### Construct Multimodal Payload

In [45]:
# 🔨 Create request body for Claude
body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1000,
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "image",
            "source": {
              "type": "base64",
              "media_type": "image/jpeg",
              "data": encoded_image
            }
          },
          {
              "type": "text", 
              "text": prompt
          }
        ]
      }
    ],
    "temperature":0.5,
    "top_k":250,
    "top_p":0.5,
    "stop_sequences":[]
})

## Execute Assessment

### Invoke Claude 3 Sonnet

In [46]:
# 🚀 Call Claude 3 Sonnet model
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
accept = 'application/json'
contentType = 'application/json'

try:
    # 📡 Send request and get response
    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    # 📋 Display assessment results
    print(response_body.get('content')[0]['text'])
    
except botocore.exceptions.ClientError as error:
    # ⚠️ Handle permission errors
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

Based on the image provided, I would assess the damage to the vehicle as follows:

Major Damage:
- The front end of the vehicle has sustained significant impact damage. Both the hood and front bumper appear to be heavily crumpled and deformed, indicating a high-energy collision.
- There is likely structural damage to the front frame rails and other components that provide structural integrity to the vehicle's front end.

Moderate Damage:
- The headlight assemblies on both sides appear to be broken or dislodged from their housings.
- Depending on the extent of the impact, there may be moderate damage to the radiator, cooling system components, and other mechanical parts located in the engine compartment.

Minor Damage:
- The windshield seems to be intact, but there may be minor cracks or chips that need to be addressed.
- The side mirrors and other exterior trim pieces may have sustained minor scratches or dents.

Repair Considerations:
Given the severity of the front-end damage, it is 

## Conclusion

In this notebook, we've demonstrated how to leverage Amazon Bedrock's Claude 3 Sonnet model to automate vehicle damage assessment for insurance claims processing. This approach brings several key advantages:

1. **Efficiency Gains**: Automating the initial damage assessment can significantly reduce the time required to process insurance claims, providing faster service to customers.

2. **Consistency**: AI-based assessments can help standardize the evaluation process across different claims, reducing variability between human adjusters.

3. **Documentation Support**: The detailed analysis generated by the model can serve as supporting documentation for claims, providing transparent explanations of damage categorization.

4. **Cost Reduction**: Streamlining the assessment process can lower operational costs for insurance companies while improving customer satisfaction through faster resolution.

The multimodal capabilities of foundation models like Claude 3 enable this solution to analyze visual information and provide structured textual outputs that align with insurance industry requirements. The model can categorize damage severity, identify affected vehicle components, suggest repair approaches, and estimate potential costs.

For production implementations, this approach could be extended by:

- Integrating with claims management systems
- Creating a feedback loop for continuous improvement
- Building comparison databases of similar damage cases
- Developing specialized prompts for different vehicle types and damage scenarios
- Implementing confidence scoring for human review escalation

This technology demonstrates how AI can transform traditional, labor-intensive processes in the insurance industry while maintaining accuracy and improving the overall claims experience.